# Drive Test Tag Generation With BERTopic
Generate tags for the written portion of the chinese driving exam using BERTopic. 

## 1. Load Data
Load data fom local database into a pandas dataframe

## 2. 